In [ ]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from models import BaseClassifier

# Gerando dados

In [2]:
# --- 4. Classificador Quadrático (QDA) e suas Variantes ---
class Modelo1(BaseClassifier):
    """
    Classificador baseado na distância de Mahalonobis.
    """
    def __init__(self):
        super().__init__()
        
    def _train(self, X_train, y_train):
        self.model_params['class_labels'] = np.unique(y_train)
        self.model_params['covariances'] = {}

        self.model_params['means'] = {}
        self.model_params['inv_covs'] = {}

        for k in self.model_params['class_labels']:
            X_k = X_train[y_train == k]

            self.model_params['means'][k] = np.mean(X_k, axis=0)
            
            cov_k = np.cov(X_k, rowvar=False)
            
            self.model_params['covariances'][k] = cov_k

            try:
                self.model_params['inv_covs'][k] = np.linalg.inv(cov_k)
            except np.linalg.LinAlgError:
                self.model_params['inv_covs'][k] = np.linalg.pinv(cov_k)

    def _predict(self, X_test):
        class_labels = self.model_params['class_labels']
        scores = np.zeros((X_test.shape[0], len(class_labels)))

        for i, k in enumerate(class_labels):
            diff = X_test - self.model_params['means'][k]
            
            scores[:, i] = np.sum((diff @ self.model_params['inv_covs'][k]) * diff, axis=1)

        predicted_indices = np.argmin(scores, axis=1)
        return class_labels[predicted_indices]

In [3]:
class Modelo2(BaseClassifier):
    """
    Classificador baseado na distância Euclidiana.
    """
    def __init__(self):
        super().__init__()
        
    def _train(self, X_train, y_train):
        self.model_params['class_labels'] = np.unique(y_train)

        self.model_params['means'] = {}

        for k in self.model_params['class_labels']:
            X_k = X_train[y_train == k]

            self.model_params['means'][k] = np.mean(X_k, axis=0)


    def _predict(self, X_test):
        class_labels = self.model_params['class_labels']
        scores = np.zeros((X_test.shape[0], len(class_labels)))

        for i, k in enumerate(class_labels):
            euclidian_diff = (X_test - self.model_params['means'][k])**2
            
            scores[:, i] = np.sum((euclidian_diff), axis=1)

        predicted_indices = np.argmin(scores, axis=1)
        return class_labels[predicted_indices]


In [ ]:
def calcular_estatisticas(y_true, y_pred):
    """
    Calcula diversas estatísticas de avaliação para um modelo de classificação.

    Args:
        y_true (array-like): Os rótulos verdadeiros.
        y_pred (array-like): As previsões do modelo.

    Returns:
        dict: Um dicionário contendo as estatísticas calculadas.
    """
    if len(y_true) != len(y_pred):
        raise ValueError("y_true e y_pred devem ter o mesmo comprimento.")

    # Convertendo para arrays numpy para facilitar a operação
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    #1 é um intruso, 0 é um não intruso
    tp = np.sum((y_pred == 1) & (y_true == 1))
    tn = np.sum((y_pred == 0) & (y_true == 0))
    fp = np.sum((y_pred == 1) & (y_true == 0))
    fn = np.sum((y_pred == 0) & (y_true == 1))

    total = len(y_true)

    # Cálculo das métricas
    accuracy = (tp + tn) / total if total > 0 else 0
    false_negative_rate = fn / (fn + tp) if (fn + tp) > 0 else 0
    false_positive_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0

    # Armazenando os resultados em um dicionário
    metrics = {
        'acuracia': accuracy,
        'taxa_falsos_negativos': false_negative_rate,
        'taxa_falsos_positivos': false_positive_rate,
        'sensibilidade': sensitivity,
        'precisao': precision
    }
    return metrics

def testar_modelo(model_class, X_train, y_train, X_test, y_test):
    model = model_class()
    model._train(X_train, y_train)
    predictions = model._predict(X_test)

    # Tratar problema de detecção de intruso
    y_test_processed = np.where(y_test == 15, 1, 0)
    predictions_processed = np.where(predictions == 15, 1, 0)

    # output_data = np.column_stack((y_test, predictions))
    # output_data_processed = np.column_stack((y_test_processed, predictions_processed))

    # full_data = np.column_stack((output_data, output_data_processed))
    # with open("resultados_modelo.csv", "w") as f:
    #     np.savetxt(f, full_data, delimiter=",", fmt="%d")

    return calcular_estatisticas(y_test_processed, predictions_processed)

def tabela_estatisticas(model_class, data: np.ndarray, Nr: int, Ptrain: float) -> pd.DataFrame:
    """
    Executa a avaliação de um classificador por múltiplas rodadas e constrói
    uma tabela com estatísticas de resumo para diversas métricas.

    Args:
        model_class: A classe do modelo a ser instanciado e avaliado.
        data (np.ndarray): O conjunto de dados completo (features e rótulo na última coluna).
        Nr (int): O número de execuções (rodadas) para a avaliação.
        Ptrain (float): O percentual de dados a serem usados para treinamento.

    Returns:
        pd.DataFrame: Uma tabela (DataFrame) com as estatísticas de resumo
                      (média, desvio padrão, min, max, mediana) para cada métrica.
    """
    n_total_samples, _ = data.shape
    X_data = data[:, :-1]
    y_data = data[:, -1].astype(int)

    n_train = round(Ptrain * n_total_samples / 100)
    n_test = n_total_samples - n_train
    
    # Dicionário para armazenar as métricas de cada rodada
    metrics_by_run = {
        'acuracia': [],
        'taxa_falsos_negativos': [],
        'taxa_falsos_positivos': [],
        'sensibilidade': [],
        'precisao': []
    }

    print(f"Executando {Nr} rodadas para a classe {model_class.__name__}...")

    for r in range(Nr):
        permutation = np.random.permutation(n_total_samples)
        train_indices = permutation[:n_train]
        test_indices = permutation[n_train:]

        X_train, y_train = X_data[train_indices], y_data[train_indices]
        X_test, y_test = X_data[test_indices], y_data[test_indices]
        
        # Testa o modelo e coleta as métricas
        run_metrics = testar_modelo(model_class, X_train, y_train, X_test, y_test)
        for key, value in run_metrics.items():
            metrics_by_run[key].append(value)
    
    # Converte o dicionário de listas em um DataFrame para facilitar a análise estatística
    df_metrics = pd.DataFrame(metrics_by_run)

    # Cria um dicionário para armazenar as estatísticas de resumo
    summary_stats = {
        'média': df_metrics.mean(),
        'desvio_padrao': df_metrics.std(),
        'mínimo': df_metrics.min(),
        'máximo': df_metrics.max(),
        'mediana': df_metrics.median()
    }
    
    # Converte as estatísticas para um DataFrame final e o transpõe
    stats_df = pd.DataFrame(summary_stats).T

    print(f"Finalizado: {model_class.__name__}. Resultados:")
    return stats_df

# Testes

In [5]:
Nr = 50     # Número de rodadas
Ptrain = 80  # Percentual de treino

In [6]:
nome_arquivo = 'modelo1.dat'
try:
    print(f"--- CARREGANDO DADOS DE '{nome_arquivo}' ---")
    D = np.loadtxt(nome_arquivo)
except FileNotFoundError:
    print(f"ERRO CRÍTICO: Arquivo '{nome_arquivo}' não encontrado.")
    exit(1)
tabela_estatisticas(Modelo1, D, Nr, Ptrain)

--- CARREGANDO DADOS DE 'modelo1.dat' ---
Executando 50 rodadas para a classe Modelo1...
Finalizado: Modelo1. Resultados:


,acuracia,taxa_falsos_negativos,taxa_falsos_positivos,sensibilidade,precisao
média,0.526286,0.474667,0.471514,0.405333,0.077726
desvio_padrao,0.221395,0.402800,0.232611,0.392120,0.117809
mínimo,0.114286,0.000000,0.096774,0.000000,0.000000
máximo,0.914286,1.000000,0.909091,1.000000,0.571429
mediana,0.542857,0.500000,0.462946,0.333333,0.044466


In [7]:
nome_arquivo = 'modelo2.dat'
try:
    print(f"--- CARREGANDO DADOS DE '{nome_arquivo}' ---")
    D = np.loadtxt(nome_arquivo)
except FileNotFoundError:
    print(f"ERRO CRÍTICO: Arquivo '{nome_arquivo}' não encontrado.")
    exit(1)
tabela_estatisticas(Modelo2, D, Nr, Ptrain)

--- CARREGANDO DADOS DE 'modelo2.dat' ---
Executando 50 rodadas para a classe Modelo2...
Finalizado: Modelo2. Resultados:


,acuracia,taxa_falsos_negativos,taxa_falsos_positivos,sensibilidade,precisao
média,0.983429,0.0,0.018122,0.980000,0.838286
desvio_padrao,0.020071,0.0,0.021980,0.141421,0.201505
mínimo,0.914286,0.0,0.000000,0.000000,0.000000
máximo,1.000000,0.0,0.093750,1.000000,1.000000
mediana,1.000000,0.0,0.000000,1.000000,0.916667
